In [ ]:
!pip install raster laspy[laszip] lazrs

In [ ]:
import laspy
import numpy as np
import os

In [ ]:
import laspy
import numpy as np
import os

def retile_laz_file(input_file: str, output_dir: str, tile_size: float) -> None:
    """
    Re-tiles a LiDAR .laz file into smaller tiles based on a defined tile size.

    Parameters
    ----------
    input_file : str
        Path to the input .laz file.
    output_dir : str
        Directory to save the tiled .laz files.
    tile_size : float
        The size of each tile in the X and Y directions.

    Returns
    -------
    None
        Saves multiple .laz files as tiles in the specified output directory.
    """
    try:
        if not os.path.exists(input_file):
            raise FileNotFoundError(f"File not found: {input_file}")

        if tile_size <= 0:
            raise ValueError("Tile size must be a positive number.")

        # Ensure output directory exists
        os.makedirs(output_dir, exist_ok=True)

        # Read the .laz file
        with laspy.open(input_file) as las_file:
            las = las_file.read()
            original_header = las.header  # Copy the original header

        # Extract point coordinates and attributes
        x, y, z = las.x, las.y, las.z

        # Get min and max values of X and Y to define grid boundaries
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Generate tile boundaries
        x_tiles = np.arange(min_x, max_x, tile_size)
        y_tiles = np.arange(min_y, max_y, tile_size)

        # Create and save tiles
        for i, x_min in enumerate(x_tiles):
            for j, y_min in enumerate(y_tiles):
                x_max, y_max = x_min + tile_size, y_min + tile_size

                # Filter points within this tile
                mask = (x >= x_min) & (x < x_max) & (y >= y_min) & (y < y_max)
                num_points = np.sum(mask)

                if num_points == 0:
                    continue  # Skip empty tiles

                # Create a new header for the tile
                tile_header = laspy.LasHeader(point_format=original_header.point_format, version=original_header.version)
                tile_header.scales = original_header.scales
                tile_header.offsets = original_header.offsets

                # Create a new LasData object with the filtered points
                tile_las = laspy.LasData(tile_header)
                tile_las.points = las.points[mask]  # Assign filtered points

                # Save the tile correctly using LasWriter
                tile_filename = os.path.join(output_dir, f"tile_{i}_{j}.laz")
                with laspy.open(tile_filename, mode="w", header=tile_header) as tile_writer:
                    tile_writer.write_points(tile_las.points)  # Use write_points() instead of write()

                print(f"Saved tile: {tile_filename}")

    except FileNotFoundError as e:
        raise FileNotFoundError(str(e))
    except ValueError as e:
        raise ValueError(str(e))
    except Exception as e:
        raise Exception(f"An error occurred: {e}")


In [ ]:
# Example of usage

# The directory of the input laz file
input_laz_file_path = "../../Datasets/Plot_1.laz"

# The directory to output the re-tiled *.laz files
output_directory = "../../Datasets/Plot_1/"

# Define the size for the new smaller tiles, in meters
tile_size = 50.0 

# Call the re-tiling function
retile_laz_file(input_laz_file_path, output_directory, tile_size)
